# Stock Data Extraction

From Yahoo Finance

Authors: Rebecca, Leni

In [1]:
import datetime
import importlib
import numpy as N
import numpy.random as NR
import matplotlib.pyplot as pyplot
import seaborn
import pandas as P
import csv
import tqdm.notebook
from pathlib import Path

import bokeh
import bokeh.plotting
import yfinance as yf

seaborn.set_style('whitegrid')

## Read Stock Data from Yahoo Finance

In [2]:
basepath = Path('datasets/Stocks')
stock_list = P.read_csv(basepath / "stocks.csv")
stock_list

,Category,Ticker
0,Tech,MSFT
1,Tech,AAPL
2,Tech,FB
3,Tech,GOOG
4,Tech,NFLX
5,Tech,AMZN
6,Tech,INTL
7,Tech,QQQ
8,Tech,QCOM
9,Energy,OXY


In [3]:
for ticker in tqdm.tqdm(stock_list['Ticker'].to_list()):
    stock = yf.Ticker(ticker)
    # Allow some rooms in order to compensate for rolling window edge
    hist = stock.history(period="max", start='2016-12-01', end='2019-08-01', interval='1d')
    
    # Must save with index
    hist.to_csv(basepath /  (ticker+'.csv'))

100%|██████████| 32/32 [00:13<00:00,  2.39it/s]


## Correction for dividend and annual interest

In [4]:
interestRate = 0.015
baseDate = P.to_datetime('2017-01-01', format='%Y-%m-%d')
def second_moment(x):
    x = N.array(x)
    if len(x) <= 1:
        return float('nan')
    x = x[:-1] - x[-1]
    return N.sqrt((x * x).mean())

def process_one(df):
    df['S'] = 0.5 * (df['Open'] + df['Close'])
    df['Date'] = df['Date'].map(lambda x: P.to_datetime(x, format='%Y-%m-%d'))
    intr = (df['Date'] - baseDate).map(lambda t:N.exp(-t.days / 365 * interestRate))
    # Stock fair value
    df['X'] = (df['S'] + df['Dividends'].cumsum()) * intr
    # Calculate the volatility in the past 7 days
    df['sigma_hat'] = df['X'].rolling(7).apply(second_moment)
    # Calculate the roll
    df['Y'] = -df['X'].diff(periods=-1) / df['sigma_hat']
    return df

#p = basepath / ('AAPL.csv')
#df = P.read_csv(p)
#process_one(df)

In [10]:
for ticker in tqdm.tqdm(stock_list['Ticker'].to_list()):
    p = basepath /  (ticker+'.csv')
    
    df = P.read_csv(p)
    df = process_one(df)
    df.to_csv(p, index=False)
    print(f"{ticker}: {len(df)}")

  6%|▋         | 2/32 [00:00<00:03,  7.80it/s]

MSFT: 669
AAPL: 669


 12%|█▎        | 4/32 [00:00<00:03,  8.20it/s]

FB: 669
GOOG: 669


 19%|█▉        | 6/32 [00:00<00:03,  8.30it/s]

...

BA: 669
LMT: 669


 94%|█████████▍| 30/32 [00:03<00:00,  8.62it/s]

RTX: 669
^IXIC: 669


100%|██████████| 32/32 [00:03<00:00,  8.52it/s]

^GSPC: 669
^DJI: 669


In [8]:
for ticker in tqdm.tqdm(stock_list['Ticker'].to_list()):
    p = basepath /  (ticker+'.csv')
    df = P.read_csv(p)
    if len(df) == 670:
        print(ticker)
        break

 59%|█████▉    | 19/32 [00:00<00:00, 218.08it/s]

BAC


# Example MSFT Ticker

In [6]:
msft = yf.Ticker("MSFT")
hist = msft.history(period="max", start='2017-01-01', end='2019-01-01', interval='1d')
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2017-01-03,58.870134,58.917012,58.251336,58.673244,20694100,0.0,0
2017-01-04,58.579488,58.832633,58.241961,58.410725,21340000,0.0,0
2017-01-05,58.307591,58.748251,58.157580,58.410725,24876000,0.0,0
2017-01-06,58.410725,59.207664,58.166958,58.917015,19922900,0.0,0
2017-01-09,58.842004,59.142030,58.635740,58.729496,20382700,0.0,0
...,...,...,...,...,...,...,...
2018-12-24,95.247732,95.530512,91.639866,91.786125,43935200,0.0,0
2018-12-26,92.770977,98.182783,91.620359,98.056015,51634800,0.0,0
2018-12-27,96.827391,98.670329,93.999601,98.660576,49498500,0.0,0


In [7]:
hist.index[1]

Timestamp('2017-01-04 00:00:00')

In [8]:
w = 12*60*60*1000
TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
plot = bokeh.plotting.figure(x_axis_type='datetime', tools=TOOLS, plot_width=1000, title="MSFT Candlestick")
plot.xaxis.major_label_orientation = N.pi / 4
plot.segment(hist.index, hist['High'], hist.index, hist['Low'], color='black')

idx = hist['Close'] > hist['Open']
plot.vbar(hist.index[idx], w, hist['Open'][idx], hist['Close'][idx], fill_color="#D5E1DD", line_alpha=0)
plot.vbar(hist.index[~idx], w, hist['Open'][~idx], hist['Close'][~idx], fill_color="#F2583E", line_alpha=0)

bokeh.plotting.output_file("analytics/stocks_msft_example.html", title="MSFT Candlestick")
bokeh.plotting.show(plot)